# Introduction to STAC Part 2:

In this tutorial, we will highlight how to create a simple STAC catalog using the python library PySTAC. We will refer back to the catalog of items we obtained from our Planet order and the corresponding analytic assets (our COGs stored in our Google Cloud Storage bucket).

### Import Dependencies

In [1]:
# Import python packages
import pystac
import os
import json
import requests
import urllib.request
import rasterio
from tempfile import TemporaryDirectory
from pathlib import Path
import typing

# Define core repository directories
ROOT_DIR = Path(os.getcwd()).parents[2]
ASSETS_DIR = os.path.join(ROOT_DIR, "assets/")

### Initialize Catalog

In [7]:
catalog = pystac.Catalog(id='overstory-stac', description='Overstory STAC Catalog.')
print(list(catalog.get_all_items()))
print(list(catalog.get_children()))

[]
[]


### Read `*_metadata.json` Files

In [45]:
# Define item id's saved in `./assets/` dir
item_ids = [
    '20210425_100412_ssc3_u0001',
    '20210425_100412_ssc3_u0002',
    '20210528_131522_ssc9_u0001',
    '20210710_133141_ssc8_u0001',
    '20210922_100732_ssc12_u0001',
    '20210922_100732_ssc12_u0002',
]
    
# Helper function to read .json files
def load_item_metadata(file):
    with open(file, "r") as f:
        data = json.load(f)
        return data

# Loop through each item
for item_id in item_ids:
    
    metadata_file = os.path.join(ASSETS_DIR, 'metadata', f'{item_id}_metadata.json') 
    metadata_data = dict(load_item_metadata(metadata_file))
    
    print(metadata_data['id'])
    print(metadata_data['properties'])
    print(metadata_data['properties']["acquired"][:-1])

20210425_100412_ssc3_u0001
{'acquired': '2021-04-25T10:04:12.75Z', 'clear_confidence_percent': 91, 'clear_percent': 93, 'cloud_cover': 0, 'cloud_percent': 0, 'ground_control_ratio': 1, 'gsd': 0.81, 'heavy_haze_percent': 0, 'item_type': 'SkySatCollect', 'light_haze_percent': 0, 'pixel_resolution': 0.5, 'provider': 'skysat', 'published': '2021-04-25T14:18:33Z', 'publishing_stage': 'finalized', 'quality_category': 'standard', 'satellite_azimuth': 35.7, 'satellite_id': 'SSC3', 'shadow_percent': 3, 'snow_ice_percent': 0, 'strip_id': 's103_20210425T100412Z', 'sun_azimuth': 145.6, 'sun_elevation': 51.7, 'updated': '2021-09-11T06:44:43Z', 'view_angle': 28.3, 'visible_confidence_percent': 78, 'visible_percent': 96}
2021-04-25T10:04:12.75
20210425_100412_ssc3_u0002
{'acquired': '2021-04-25T10:04:12.75Z', 'clear_confidence_percent': 91, 'clear_percent': 93, 'cloud_cover': 0, 'cloud_percent': 0, 'ground_control_ratio': 1, 'gsd': 0.8, 'heavy_haze_percent': 0, 'item_type': 'SkySatCollect', 'light_ha

### Create STAC Items

In [38]:
# Import python packages
from rasterio.warp import calculate_default_transform
from shapely.geometry import Polygon, mapping
from datetime import datetime
from itertools import islice

In [ ]:
# Define helper function to get STAC item
def create_STAC_Item(tiff_path, metadata_json):
    """Creates a STAC item.
    
    Args:
        tiff_path (str): path to the .tif file
        metadata_json (str): 
        
    Returns:
        
    """

    with rasterio.open(tiff_path) as sample_cog:
        
        bounds = sample_cog.bounds
        src_crs = sample_cog.crs
        dst_crs = 'EPSG:4326'  # EPSG identifier for WGS84 coordinate system used by the geojson format
        
        left, bottom, right, top = rasterio.warp.transform_bounds(sample_cog.crs, dst_crs, *bounds)
        bbox = [left, bottom, right, top]
        
        # Create geojson feature
        geom = mapping(Polygon([
          [left, bottom],
           [left, top],
           [right, top],
           [right, bottom]
        ]))
        
        time_acquired = datetime.strptime(metadata_json["properties"]["acquired"][:-1], '%Y-%m-%dT%H:%M:%S.%f')
        
        # Instantiate pystac item
        item = pystac.Item(id=metadata_json["id"],
                 geometry=geom,
                 bbox=bbox,
                 datetime = time_acquired,
                 properties={
                 })

        # Use Planet metadata.json to add some common metadata to the STAC item
        metadata_properties = metadata_json["properties"]

        # Enable item extensions
        item.ext.enable('eo')
        item.ext.enable('view') 
        item.ext.enable('projection')
        
        for key, value in islice(metadata_properties.items(), 1, None):

            # Add some common metadata for the item not included in the core item specification
            if(key == 'gsd'):
                item.common_metadata.gsd = value
            
            # View Geometry Extension 
            if (key == 'sun_azimuth'):
                item.ext.view.sun_azimuth = value
            if (key == 'sun_elevation'):
                item.ext.view.sun_elevation = value
            
            # Electro Optical Extension - 
            if(key == 'cloud_cover'):
                item.ext.eo.cloud_cover = value
           
           # Projection Extension
            if(key == 'epsg_code'):
                item.ext.projection.epsg = value

        # Tuple containing spatial and temporal extent information to use later in this tutorial
        item_extent_info =  (bbox, geom, time_acquired)
     
    # Returns a list containing the PySTAC Item object and a tuple 
    # holding the bounding box, geojson polygon, and date the item was acquired
    return item, (item_extent_info)

In [ ]:

def create_STAC_Items(metadata_folder_name, planet_order_id, item_type, item_ids, storage_bucket_name):
    """ Create STAC Items.
    
    Args:
        metadata_folder_name:
        planet_order_id:
        item_type:
        item_ids:
        storage_bucket_name: 
    
    Returns:
        
    """

    # Store metadata 
    store_item_metadata(order_id, metadata_folder_name, item_type , item_ids)
    metadata_directory = metadata_folder_name + '/' + order_id + '/' + item_type
    metadata_files = sorted(Path(metadata_directory).glob('*'), reverse=True)

    urls = []
    urls = sorted([storage_bucket_name + item_id + '_pansharpened_udm.tif' for item_id in item_ids], reverse=True)

    # empty list to store STAC items
    stac_items = []
    
    for asset_url, item_metadata in zip(urls, metadata_files):
        m = load_item_metadata(item_metadata)

        item, extent = create_STAC_Item(asset_url, m)
        item.add_asset(
              key='analytic',
              asset=pystac.Asset(
                  href=asset_url,
                  title= "4-Band Analytic",
                  # indicate it is a cloud optimized geotiff
                  media_type=pystac.MediaType.COG,
                  roles=([
                    "analytic"
                  ])
              )
        ) 
        stac_items.append((item, extent))
    return stac_items

Add the STAC items to a local directory

In [ ]:

item_type = 'PSScene4Band'
metadata_folder = 'catalog_metadata'
storage_bucket_name = os.path.join(ASSETS_DIR, 'cog')

stac_items = create_STAC_Items(
    metadata_folder_name=metadata_folder, 
    planet_order_id=order_id, 
    item_type=item_type, 
    item_ids=item_ids, 
    storage_bucket_name=storage_bucket_name
)

### Add Items to STAC Catalog

In [ ]:
for index, item in enumerate(stac_items):
    catalog.add_item(item[0])
    print(json.dumps(item[0].to_dict(), indent=4))